<a href="https://colab.research.google.com/github/Elshraby/DeepAerofoil/blob/main/27_May_MypromptwChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import files

# Upload airfoil and turbine design data
uploaded = files.upload()

# Load files
airfoil_df = pd.read_csv("airfoil_data.csv")  # Angle of attack (deg), Cl, Cd
turbine_df = pd.read_csv("turbine_data.csv")  # All turbine parameters including Cp

In [ ]:
def bem_power_coefficient(row, cl_interp, cd_interp):
    B = row['Number_of_Blades']
    R = row['Diameter'] / 2
    H = row['Height']
    c = row['Chord_Length']
    U_inf = row['Freestream_Velocity']
    TSR = row['Tip_Speed_Ratio']

    rho = 1.225  # kg/m^3
    n_segments = 20
    dtheta = 2 * np.pi / n_segments
    power = 0.0

    for i in range(n_segments):
        theta = i * dtheta
        omega = TSR * U_inf / R
        V_rel_x = U_inf * np.cos(theta)
        V_rel_y = omega * R + U_inf * np.sin(theta)
        V_rel = np.sqrt(V_rel_x**2 + V_rel_y**2)
        alpha = np.arctan2(V_rel_y, V_rel_x) * 180 / np.pi  # in degrees

        Cl = cl_interp(alpha)
        Cd = cd_interp(alpha)
        dL = 0.5 * rho * V_rel**2 * c * Cl
        dD = 0.5 * rho * V_rel**2 * c * Cd

        dT = dL * np.cos(np.radians(alpha)) - dD * np.sin(np.radians(alpha))
        dP = dT * omega * R
        power += dP * dtheta

    P_available = 0.5 * rho * U_inf**3 * (np.pi * R**2)
    Cp = power / P_available
    return Cp

In [ ]:
from scipy.interpolate import interp1d

cl_interp = interp1d(airfoil_df['Angle'], airfoil_df['Cl'], fill_value='extrapolate')
cd_interp = interp1d(airfoil_df['Angle'], airfoil_df['Cd'], fill_value='extrapolate')

In [ ]:
turbine_df['Cp_BEM'] = turbine_df.apply(lambda row: bem_power_coefficient(row, cl_interp, cd_interp), axis=1)

In [ ]:
# Select inputs and output
features = ['Number_of_Blades', 'Chord_Length', 'Height', 'Diameter', 'Freestream_Velocity', 'Tip_Speed_Ratio', 'Cp_BEM']
target = 'Power_Coefficient'

X = turbine_df[features].values
y = turbine_df[target].values.reshape(-1, 1)

# Scale data
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
class CpPredictor(nn.Module):
    def __init__(self):
        super(CpPredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(7, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
model = CpPredictor()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 500
losses = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.grid(True)
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predictions = scaler_y.inverse_transform(predictions.numpy())
    y_true = scaler_y.inverse_transform(y_test.numpy())

mse = np.mean((predictions - y_true)**2)
print(f"Test MSE: {mse:.6f}")

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_true, predictions, alpha=0.7)
plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'r--')
plt.xlabel("True Cp")
plt.ylabel("Predicted Cp")
plt.title("Prediction vs Actual")
plt.grid(True)
plt.axis('equal')
plt.show()